In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import re
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bertscore


In [2]:
import pandas as pd
import requests

url_english = "https://drive.google.com/uc?export=download&id=1J5zhBA3qvKFPbADziKRv1xBQ9A_jMdRs"
text = requests.get(url_english).text
lines = text.splitlines()
df_english = pd.DataFrame({"text": lines})
url_indo = "https://drive.google.com/uc?export=download&id=1_jMC6ImrPz2KJzj4aWFdTiPS3_2-DcG6"
text = requests.get(url_indo).text
lines = text.splitlines()
df_indo = pd.DataFrame({"text": lines})


In [3]:
df_indo.head(10)

,text
0,“Saat ini ada mencit umur 4 bulan nondiabetes ...
1,"Dr. Ehud Ur, profesor kedokteran di Dalhousie ..."
2,"Seperti halnya ahli-ahli lain, dia merasa skep..."
3,"Sara Danius, sekretaris permanen Komite Nobel ..."
4,"""Danius berkata, ""Saat ini kita tidak melakuka..."
5,"Sebelumnya, CEO Ring, Jamie Siminoff, berkomen..."
6,Ia melakukan pembuatan bel pintu dengan teknol...
7,Siminoff mengatakan penjualan meningkat setela...
8,"Di akhir tahun 2017, Siminoff muncul di salura..."
9,Ring juga menyelesaikan sebuah tuntutan hukum ...


In [4]:
device = "cuda"
model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map=device
).eval()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
FEW_SHOT = """
English: In 2010, South Korea experienced a particularly cold winter.
Indonesian: Pada 2010, Korea Selatan mengalami musim dingin yang tidak biasa.

English: People couldn't activate their smartphones while wearing gloves, 
so they began wielding snack sausages—causing one company to see a 40% rise in sausage sales.
Indonesian: Orang-orang tidak dapat menyalakan ponsel pintar mereka saat mengenakan sarung tangan, 
jadi mereka mulai menggunakan camilan sosis, yang menyebabkan satu perusahaan mengalami kenaikan penjualan sosis sebesar 40%.
""".strip()

def prompt_zero_shot(text):
    return f"""Translate to Indonesian:

English: "{text}"
Indonesian:""".strip()

def prompt_few_shot(text):
    return f"""Translate English to Indonesian following the examples.

{FEW_SHOT}

English: "{text}"
Indonesian:""".strip()

def prompt_role_based(text):
    return f"""You are a professional bilingual educator who creates flashcards.
Translate the English sentence into natural Indonesian.
Use complete sentences. No explanation. Output ONLY Indonesian.

English: "{text}"
Indonesian:""".strip()

def prompt_cot_guarded(text):
    return f"""Translate the English sentence into Indonesian.
First think through the meaning step-by-step to ensure accuracy.
Then provide the final Indonesian translation ONLY in the last line, prefixed with "Indonesian:".

English: "{text}"

Reasoning:
""".strip()

def prompt_back_translation(text):
    return f"""Step 1: Translate the English sentence to Indonesian.
Step 2: Translate that Indonesian back to English to verify consistency.
Step 3: Output ONLY the final Indonesian translation in the last line, prefixed with "Final Indonesian:".

English: "{text}"

Indonesian:
""".strip()


In [ ]:
import re

def extract_back_translation(gen):
    if not isinstance(gen, str):
        return ""

    m = re.search(r'Final Indonesian\s*:\s*"([^"]+)"', gen, flags=re.I)
    if m:
        return m.group(1).strip()

    m2 = re.search(r'Indonesian\s*:\s*"([^"]+)"', gen, flags=re.I)
    if m2:
        return m2.group(1).strip()
    m3 = re.search(r'"([^"]+)"', gen)
    if m3:
        return m3.group(1).strip()
    return gen.split("\n")[0].strip()

In [ ]:
def strip_wrappers(s: str):
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = s.lstrip("['").rstrip("']")
    s = s.strip()
    s = s.strip('"').strip("'").strip()

    return s

In [8]:
def generate_text(prompt, max_new_tokens=60):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.0,
            do_sample=False
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def extract_translation(full_output, prompt, method):
    gen = full_output.replace(prompt, "").strip()

    gen = re.sub(r"\r\n?", "\n", gen).strip()

    if method in ["zero_shot", "few_shot", "role_based"]:
        lines = [l.strip() for l in gen.split("\n") if l.strip()]
        if not lines:
            return ""
        m = re.match(r'^(Indonesian|Indonesia|Terjemahan|Translation)\s*:\s*(.*)$', lines[0], re.I)
        return strip_wrappers((m.group(2).strip() if m else lines[0]))
    if method == "back_translation":
        return strip_wrappers(extract_back_translation(gen))
    return gen.split("\n")[0].strip()

def generate_only_translation(prompt, method, max_new_tokens=160):
    full_output = generate_text(prompt, max_new_tokens=max_new_tokens)
    return extract_translation(full_output, prompt, method)

In [9]:
smooth = SmoothingFunction().method1
rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

def compute_bleu(ref, hypo):
    return sentence_bleu([ref.split()], hypo.split(), smoothing_function=smooth)

def compute_rougeL(ref, hypo):
    return rouge.score(ref, hypo)["rougeL"].fmeasure

def compute_bertscore(ref, hypo):
    P, R, F1 = bertscore([hypo], [ref], lang="id")
    return F1[0].item()

In [ ]:
def run_single_pipeline(df_en, df_id, method_name):
    methods = {
        "zero_shot": prompt_zero_shot,
        "few_shot": prompt_few_shot,
        "role_based": prompt_role_based,
        "back_translation": prompt_back_translation,
    }

    if method_name not in methods:
        raise ValueError(f"Invalid method. Choose from: {list(methods.keys())}")

    method_func = methods[method_name]

    all_outputs = []
    bleu_list, rouge_list, bert_list = [], [], []

    print(f"\n=== RUNNING {method_name.upper()} (NO BATCHING) ===")

    for i in tqdm(range(len(df_en))):

        en = df_en.loc[i, "text"]
        gt = df_id.loc[i, "text"]

        prompt = method_func(en)
        full_output = generate_text(prompt, max_new_tokens=60)
        pred = extract_translation(full_output, prompt, method_name)

        all_outputs.append(pred)

        bleu_list.append(compute_bleu(gt, pred))
        rouge_list.append(compute_rougeL(gt, pred))
        bert_list.append(compute_bertscore(gt, pred))

        torch.cuda.empty_cache()

    # aggregate
    scores = {
        "bleu": float(np.mean(bleu_list)),
        "rougeL": float(np.mean(rouge_list)),
        "bertscore": float(np.mean(bert_list)),
    }

    return all_outputs, scores

In [ ]:
tokenizer.padding_side = "left"
def generate_batch(prompts, max_new_tokens=60):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            use_cache=False,   
        )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
def run_batched_pipeline(df_en, df_id, method_name, batch_size=4):
    methods = {
        "zero_shot": prompt_zero_shot,
        "few_shot": prompt_few_shot,
        "role_based": prompt_role_based,
        "back_translation": prompt_back_translation,
    }

    if method_name not in methods:
        raise ValueError(f"Invalid method. Choose from: {list(methods.keys())}")

    method_func = methods[method_name]

    all_outputs = []
    bleu_list, rouge_list, bert_list = [], [], []

    print(f"\n=== RUNNING {method_name.upper()} | BATCH SIZE = {batch_size} ===")

    num_samples = len(df_en)

    for start in tqdm(range(0, num_samples, batch_size)):
        end = min(start + batch_size, num_samples)

        en_batch = [df_en.loc[i, "text"] for i in range(start, end)]
        gt_batch = [df_id.loc[i, "text"] for i in range(start, end)]

        prompts = [method_func(en) for en in en_batch]

        raw_outputs = generate_batch(prompts, max_new_tokens=60)

        preds = [
            extract_translation(raw_outputs[j], prompts[j], method_name)
            for j in range(len(raw_outputs))
        ]

        all_outputs.extend(preds)

        for j in range(len(preds)):
            bleu_list.append(compute_bleu(gt_batch[j], preds[j]))
            rouge_list.append(compute_rougeL(gt_batch[j], preds[j]))
            bert_list.append(compute_bertscore(gt_batch[j], preds[j]))

        torch.cuda.empty_cache()

    scores = {
        "bleu": float(np.mean(bleu_list)),
        "rougeL": float(np.mean(rouge_list)),
        "bertscore": float(np.mean(bert_list)),
    }

    return all_outputs, scores

In [13]:
def evaluate_all_methods(df_en, df_id, batch_size=4):
    methods = ["zero_shot", "few_shot", "role_based", "back_translation"]

    results = {}

    for m in methods:
        print(f"\n========== Evaluating {m.upper()} ==========")
        outputs, scores = run_batched_pipeline(df_en, df_id, m, batch_size=batch_size)
        results[m] = scores
        print(scores)

    return results

In [ ]:
from datasets import load_dataset
import pandas as pd
dset = load_dataset("SEACrowd/identic", trust_remote_code=True)
train = dset["train"]
pairs = []
for i in train:
    pairs.append({
        "english": i["en_sentence"],
        "indonesian": i["id_sentence"]
    })
df = pd.DataFrame(pairs)
df.to_csv("identic_en_id.csv", index=False, encoding="utf-8")

print("Saved to identic_en_id.csv")
print()
print(df.head(5))

Saved to identic_en_id.csv

                                             english  \
0  Delhi ropes in monkeys for Commonwealth Games ...   
1  Delhi authorities have deployed a contingent o...   
2  At least 10 langurs have been on duty outside ...   
3  Delhi civic authorities have 28 langurs and 10...   
4  Thousands of monkeys roam Delhi, mostly around...   

                                          indonesian  
0                  Kera untuk amankan pesta olahraga  
1  Pemerintah kota Delhi mengerahkan monyet untuk...  
2  Beberapa laporan menyebutkan setidaknya 10 mon...  
3  Pemkot Delhi memiliki 28 monyet dan berencana ...  
4  Jumlah monyet di ibukota India mencapai ribuan...  


In [14]:
results = evaluate_all_methods(df_english, df_indo, batch_size=6)


========== Evaluating ZERO_SHOT ==========

=== RUNNING ZERO_SHOT | BATCH SIZE = 6 ===


100%|██████████| 169/169 [34:07<00:00, 12.12s/it]


{'bleu': 0.21997517773039588, 'rougeL': 0.5819825048123606, 'bertscore': 0.8749076050260792}

========== Evaluating FEW_SHOT ==========

=== RUNNING FEW_SHOT | BATCH SIZE = 6 ===


100%|██████████| 169/169 [39:42<00:00, 14.10s/it]


{'bleu': 0.21584506595623718, 'rougeL': 0.5721688444213027, 'bertscore': 0.8644501088047216}

========== Evaluating ROLE_BASED ==========

=== RUNNING ROLE_BASED | BATCH SIZE = 6 ===


100%|██████████| 169/169 [35:18<00:00, 12.53s/it]


{'bleu': 0.2143740261448708, 'rougeL': 0.5801238872723201, 'bertscore': 0.8742129639556757}

========== Evaluating BACK_TRANSLATION ==========

=== RUNNING BACK_TRANSLATION | BATCH SIZE = 6 ===


100%|██████████| 169/169 [35:04<00:00, 12.45s/it]

{'bleu': 0.2003157764682237, 'rougeL': 0.5421106435793569, 'bertscore': 0.858647780279397}


In [15]:
# outputs_zero, scores_zero = run_single_pipeline(df_english, df_indo, "zero_shot")
# print(scores_zero)

In [16]:
# outputs_few, scores_few = run_single_pipeline(df_english, df_indo, "few_shot")
# print(scores_few)

In [17]:
# outputs_role, scores_role = run_single_pipeline(df_english, df_indo, "role_based")
# print(scores_role)

In [18]:
# outputs_back, scores_back = run_single_pipeline(df_english, df_indo, "back_translation")
# print(scores_back)

In [19]:
# df_one_en = df_english.iloc[[0]]
# df_one_id = df_indo.iloc[[0]]

# outputs, scores = run_batched_pipeline(df_one_en, df_one_id, "zero_shot", batch_size=1)
# print(outputs)
# print(scores)